# Exploring the Relationship Between Reddit Sentiment and Stock Price Movements

# 1. Introduction & Hypothesis

**Research Question**  
Is the sentiment in stock-related Reddit posts correlated with short-term stock price movements?

This project investigates the relationship between social media sentiment and financial markets by analyzing one year of posts from stock-focused subreddits about three major tech stocks: NVIDIA, Tesla, and Google. The goal is to determine if statistically significant correlations exist between online discussions and stock performance.

**Initial Hypothesis**  
There is a statistically significant positive correlation between weekly Reddit sentiment scores and weekly stock returns for the selected technology stocks.

# 2. Methodology Overview

To test this hypothesis, we will:

1. **Collect Data**  
   Gather historical stock price data for NVDA, TSLA, and GOOGL and collect Reddit posts mentioning these stocks from relevant subreddits over the same time period.

2. **Process Data**  
   - Clean the text data and address data quality issues
   - Calculate sentiment scores for each post using two NLP libraries: TextBlob and VADER
   - Calculate weekly stock returns from stock price data
   - Combine individual post sentiments into weekly averages
   - Merge the weekly sentiment and stock return datasets

3. **Analyze & Visualize**  
   Use descriptive statistics, visualizations, and correlation analysis to examine the relationship between weekly sentiment and stock returns.

4. **Critically Assess**  
   Evaluate the strength, statistical significance, and practical importance of any relationships found, while discussing methodological limitations.

## 2.1 Data Source Justification

**A. Subreddit Selection**  
The Reddit data was collected from five stock-focused communities: "r/wallstreetbets", "r/stocks", "r/investing", "r/StockMarket", and "r/StocksAndTrading".

* **Subreddit Justification**  
These communities focus specifically on stock discussion and investing, providing direct access to market sentiment. General news or technology forums were avoided because their discussions may not be investment-oriented.

* **Excluded Subreddits**  
Specialized communities like "r/Options" (focused on short-term options trading) and "r/SecurityAnalysis" (focused on long-term valuation) were excluded because their specific contexts would introduce different types of signals, making it harder to capture general retail investor sentiment.

* **Note on variation**  
The selected subreddits represent different investor perspectives, from the speculative nature of "r/wallstreetbets" to the long-term focus of "r/investing". This variation is acknowledged as part of our dataset.

**B. Stock Selection**  
The analysis focuses on **NVIDIA (NVDA), Tesla (TSLA), and Alphabet (GOOGL)** for three reasons:

1. **High Discussion Volume**  
   These are among the most discussed companies on social media, ensuring enough Reddit posts for reliable analysis.

2. **Varied Volatility**  
   NVDA and TSLA have significant price volatility, while GOOGL is more stable. This mix helps test if sentiment correlations work better with volatile or more stable stocks.

3. **Sector Diversity**  
   While all are technology companies, they represent different areas (semiconductors, electric vehicles, internet services), allowing us to see if sentiment relationships hold across different business models.

**C. Timeframe**  
The analysis covers one year of data. This timeframe was chosen because the Reddit API efficiently provides posts from up to one year ago. Stock price data was collected for the same period to ensure alignment. This gives us about one year of data points per stock, which is sufficient for correlation analysis.

**Note:** Both stock price data and Reddit posts contain data until **October 28, 2025**

## 2.2 Success Criteria

This project will be successful if it can:
- Identify statistically significant correlation coefficients between weekly sentiment scores and weekly stock returns
- Provide a thoughtful discussion about the practical importance of any findings, acknowledging that correlation doesn't mean causation

## 2.3 Weekly Analysis Approach

After considering daily and weekly options, I chose weekly aggregation because daily sentiment data has too much random noise, while weekly data smooths out these short-term fluctuations, while still captures social media's fast-moving nature. This approach provides enough data points for reliable statistical testing while keeping sentiment and returns properly aligned.

**How it works:**
- Stock returns: Closing price percentage change from Friday to Friday
- Reddit sentiment: Average sentiment from posts from Saturday to Friday
- Only complete weeks are included to ensure correct one week stock returns

## 2.4 Important Limitations

Several limitations should be kept in mind when interpreting the results:

* **Correlation vs. Causation**  
  This study can find correlations but cannot prove that sentiment causes price moves. Price movements could influence sentiment, or both are driven by other factors like news events.

* **Other Influencing Factors**  
  Stock prices are affected by many things we're not measuring, like earnings reports, economic data, and world events. Our analysis only looks at sentiment.

* **Reddit User Bias**  
  The sentiment comes only from Reddit users, who may not represent all investors. Findings apply specifically to this group.

## 2.5 Analysis Methods

**Descriptive Statistics**  
I will use arithmetic means for all summary statistics to maintain consistency when reporting post volumes, sentiment scores, and other metrics.

**Correlation Analysis**  
Pearson correlation coefficients will measure relationships between sentiment scores and stock returns, which is appropriate for continuous variables and aligns with our hypothesis testing approach.

**Sentiment Analysis**
Two Natural Language Processing (NLP) libraries are used to calculate sentiment scores:

- VADER  
  Chosen because it is specifically attuned to sentiments expressed in social media, capturing nuances like slang and emojis.

- TextBlob  
  Chosen as a well-established, general-purpose sentiment analysis tool to provide a baseline for comparison.

Using both libraries allows to assess whether the relationship with stock returns is robust across different methods of measuring sentiment or is specific to one tool's approach.

# 3. Packages Installation and Imports

**Note:** The following cell installs the necessary Python packages. This is only required once per runtime environment.

In [ ]:
# Install required packages
!pip install yfinance pandas numpy matplotlib seaborn vaderSentiment textblob python-dotenv statsmodels praw

In [ ]:
# Utils
import os
import numpy as np
import pandas as pd
from scipy.stats import skew, pearsonr, norm
from dotenv import load_dotenv
from datetime import datetime, timedelta

# APIs & Web
import praw
import yfinance as yf

# NLP & Sentiment
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Jupyter Notebook Magic for inline plots
%matplotlib inline

load_dotenv() # Loads REDDIT_CLIENT_ID, REDDIT_CLIENT_SECRET, REDDIT_USER_AGENT

print("=== IMPORTS FINISHED ===")

# 4. Data Collection & Preparation

This section implements the data collection pipeline described in Section 2, gathering and preparing both stock market data and Reddit sentiment data for correlation analysis.

### 4.0.1 Configuration

In [ ]:
# Configuration
data_dir = './data'

stocks_config = {
    'NVDA': {
        'color': 'green',
        'keywords': ['nvidia', 'nvda', "jensen huang", "rtx", "geforce"],
    },
    'TSLA': {
        'color': 'orange',
        'keywords': ['tesla', 'tsla', 'elon', 'musk'],
    },
    'GOOGL': {
        'color': 'blue',
        'keywords': ['google', 'googl', 'alphabet', 'gemini', "android"],
    }
}

subreddits_list = ['wallstreetbets', 'stocks', 'investing', 'StockMarket', 'StocksAndTrading']

# Create data directory if it doesn't exist
os.makedirs(data_dir, exist_ok=True)

print(" === CONFIGURATION FINISHED ===")

## 4.1 Stock Data Collection

In [ ]:
def load_stocks_data_from_files(tickers=stocks_config.keys()):
    """
    Load stock price data from saved CSV files.
    
    Args:
        tickers (list): List of stock ticker symbols.
        
    Returns:
        DataFrame: Combined DataFrame with all stocks
    """
    stock_data_list = []
    
    print("Loading saved stock data...")
    for ticker in tickers:
        file_path = os.path.join(data_dir, f"stock_data_{ticker}.csv")
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Stock data file not found: {file_path}")
        
        try:
            df = pd.read_csv(file_path, parse_dates=['date'], index_col='date')
            stock_data_list.append(df)
            print(f"✓ Loaded data for {ticker} (Shape: {df.shape})")
            
        except Exception as e:
            print(f"✗ Loading stock data for {ticker} failed: {e}")
            raise
    
    # Combine all stocks into one DataFrame
    combined_data = pd.concat(stock_data_list, ignore_index=False)
    return combined_data


def download_and_save_stocks_data(tickers=stocks_config.keys()):
    """
    Download fresh stock data and save to CSV files.
    
    Args:
        tickers (list): List of stock ticker symbols.
        
    Returns:
        DataFrame: Combined DataFrame with all stocks
    """
    stock_data_list = []
    
    print("Downloading fresh stock data...")
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)
    
    for ticker in tickers:
        try:
            # Download data
            df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True, progress=False)
            print(df)
            if df.empty:
                print(f"✗ No data downloaded for {ticker}")
                continue

            # Clean up column structure
            df.columns = df.columns.droplevel(1) # Remove stock row from header
            df = df.reset_index()
            df.columns = ['date', 'close', 'high', 'low', 'open', 'volume']
            df = df.set_index('date')
            df['ticker'] = ticker
            
            stock_data_list.append(df)
            print(f"✓ Downloaded data for {ticker} (Shape: {df.shape})")
            
            # Save the individual stock data
            file_path = os.path.join(data_dir, f"stock_data_{ticker}.csv")
            df.to_csv(file_path)
            print(f"  Saved to {file_path}")
            
        except Exception as e:
            print(f"✗ Downloading stock data for {ticker} failed: {e}")
    
    # Combine all stocks into one DataFrame
    combined_data = pd.concat(stock_data_list, ignore_index=False)
    return combined_data


print("=== STOCK DATA LOADING ===")
try:
    stocks_data_raw = load_stocks_data_from_files()
    data_source = "saved files"
    print("=== STOCK DATA LOADED ===")
    print("\n--- Stock Data Info ---")
    print(stocks_data_raw.info())
    print("\n--- Stock Data Sample ---")
    print(stocks_data_raw.head(5))
    
except FileNotFoundError as e:
    print(f"⚠ {e}")
    print("Note: Download functionality disabled")
    # stocks_data_raw = download_and_save_stocks_data()
    # data_source = "fresh download"
    raise

## 4.2 Reddit Data Collection

**Reddit Data Sampling Strategy**  
Reddit posts were collected using the PRAW library.
By setting sort="top" and time_filter="year", I collected the most upvoted posts from the past year that matched the keywords.

**Rationale**  
This approach prioritizes impactful discourse. Highly upvoted posts represent sentiment seen and endorsed by a larger portion of the subreddit community, providing a stronger signal of collective sentiment than a simple chronological feed, which would be dominated by low-engagement posts.

In [ ]:
def load_reddit_posts_from_file():
    """
    Load Reddit posts from saved CSV file.
    
    Returns:
        DataFrame: Reddit posts data
        
    Raises:
        FileNotFoundError: If the Reddit data file is missing
    """
    file_path = f"{data_dir}/reddit_posts.csv"
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Reddit data file not found: {file_path}")
    
    try:
        posts_df = pd.read_csv(file_path, parse_dates=['datetime'])
        print("✓ Loaded saved Reddit data")
        return posts_df
    except Exception as e:
        print(f"✗ Loading Reddit data failed: {e}")
        raise

def download_reddit_posts(stocks_config=stocks_config, subreddits=subreddits_list):
    """
    Download fresh Reddit posts from stock-focused subreddits.
    
    Args:
        stocks_config (dict): Contains stock tickers and configurations
        subreddits (list): List of stock-focused subreddits
        
    Returns:
        DataFrame: Downloaded Reddit posts data
    """
    reddit_posts = []
    seen_posts = set()  # Tracks post IDs to avoid duplicates

    try:
        reddit = praw.Reddit(
            client_id=os.getenv('REDDIT_CLIENT_ID'),
            client_secret=os.getenv('REDDIT_CLIENT_SECRET'), 
            user_agent=os.getenv('REDDIT_USER_AGENT')
        )
        print("✓ Reddit API connected successfully")
    except Exception as e:
        print(f"✗ Reddit API failed: {e}")
        return pd.DataFrame()

    for ticker in stocks_config.keys():
        search_query = " OR ".join(stocks_config[ticker]['keywords'])
            
        for subreddit_name in subreddits:
            print(f"Collecting posts for {ticker} from r/{subreddit_name}...")
            try:
                subreddit = reddit.subreddit(subreddit_name)
                subreddit_posts = subreddit.search(
                    query=search_query,
                    sort="top",
                    time_filter="year"
                )
                
                for post in subreddit_posts:
                    # Skip duplicates
                    if post.id in seen_posts:
                        continue
                    seen_posts.add(post.id)
                    
                    content = (post.title + ' ' + (post.selftext or '')).lower()
                    
                    # Check for stock mentions
                    stock_mentions = []
                    for stock, config in stocks_config.items():
                        keywords = config['keywords']
                        if any(keyword in content for keyword in keywords):
                            stock_mentions.append(stock)
                    
                    if stock_mentions:
                        post_data = {
                            'id': post.id,
                            'datetime': datetime.fromtimestamp(post.created_utc),
                            'subreddit': subreddit_name,
                            'stock': ', '.join(stock_mentions),
                            'title': post.title,
                            'content': post.selftext,
                            'score': post.score,
                        }
                        reddit_posts.append(post_data)
                    
            except Exception as e:
                print(f"Error with r/{subreddit_name}: {e}")
                continue

    posts_df = pd.DataFrame(reddit_posts)
    
    if not posts_df.empty:
        posts_df.to_csv(f"{data_dir}/reddit_posts.csv", index=False)
        print(f"✓ Saved {len(posts_df)} Reddit posts to CSV")
    else:
        print("No Reddit posts collected")
    
    return posts_df


print("=== REDDIT DATA LOADING ===")
try:
    reddit_posts_raw = load_reddit_posts_from_file()
    data_source = "saved files"
    print("=== REDDIT DATA LOADED ===")
    print("\n--- Reddit Posts Info ---")
    print(reddit_posts_raw.info())
    print("\n--- Reddit Posts Sample ---")
    print(reddit_posts_raw.head(5))
    
except FileNotFoundError as e:
    print(f"⚠ {e}")
    print("Note: Download functionality disabled")
    # reddit_posts_raw = download_reddit_posts()
    # data_source = "fresh download"
    raise

## 4.3 Comprehensive Data Inspection & Validation

This section provides a foundational understanding of our datasets' structure, quality, and temporal alignment before proceeding with analysis. We examine each data source individually and then assess their compatibility.

### 4.3.1 Stock Data Fundamentals

We begin by visualizing the price trends and key statistics for the three target stocks. Plotting normalized prices allows for direct comparison of performance and volatility despite different price levels.

In [ ]:
# aligns close values in horizontal table with index and ticker
pivot_data = stocks_data_raw.pivot(columns='ticker', values='close')
normalized_pivot = (pivot_data / pivot_data.iloc[0]) * 100

plt.figure(figsize=(12, 6))
for ticker in normalized_pivot.columns:
    plt.plot(normalized_pivot.index, normalized_pivot[ticker], 
             label=ticker, linewidth=2, color=stocks_config[ticker]['color'])

plt.title("Stock Price Performance (Normalized to Starting Value = 100)", fontsize=14, fontweight='bold')
plt.ylabel("Normalized Price (%)")
plt.xlabel("Date")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n--- Stocks Fundamentals Summary ---")
for ticker in stocks_config.keys():
    stock_data = stocks_data_raw[stocks_data_raw['ticker'] == ticker]
    
    # Key metrics
    price_range = stock_data['close'].max() - stock_data['close'].min()
    volatility = stock_data['close'].pct_change().std() * 100
    total_return = ((stock_data['close'].iloc[-1] - stock_data['close'].iloc[0]) / stock_data['close'].iloc[0]) * 100
    avg_volume = stock_data['volume'].mean()
    
    print(f"\n{ticker}:")
    print(f"  Time Period: {stock_data.index.min().date()} to {stock_data.index.max().date()}")
    print(f"  Trading Days: {len(stock_data)}")
    print(f"  Missing Values: {stock_data.isnull().sum().sum()}")
    print(f"  Price Range: ${stock_data['close'].min():.2f} - ${stock_data['close'].max():.2f} (${price_range:.2f} spread)")
    print(f"  Starting Price: ${stock_data['close'].iloc[0]:.2f}")
    print(f"  Ending Price: ${stock_data['close'].iloc[-1]:.2f}")
    print(f"  Total Return: {total_return:+.2f}%")
    print(f"  Daily Volatility: {volatility:.2f}%")

### Initial Observations

- **Bullish Market Context**  
  All three stocks showed strong positive returns (+43% to +75%) over the analysis period, indicating a generally bullish market environment for technology stocks.

- **Volatility Comparison**  
  The normalized plot clearly shows the different volatility profiles, with TSLA exhibiting the largest swings (4.27% daily volatility) and GOOGL appearing most stable (2.06% daily volatility), confirming our initial stock selection rationale.

- **Performance Divergence**  
  The stocks show distinct performance patterns over the year, with significant divergence in returns (TSLA +75% vs NVDA +43%). This variation is ideal for testing our hypothesis, as we have different "signal" strengths to analyze.

- **Data Quality**  
  All three stocks show complete data with no missing values and consistent trading days, providing a solid foundation for analysis.

### 4.3.2 Reddit Data Composition & Quality

Next, we explore the structure and distribution of the Reddit data. This helps understand the volume, source, and potential quality issues of our sentiment data.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Raw post distribution by stock
stock_counts = reddit_posts_raw['stock'].value_counts()
stock_counts.plot(kind='bar', ax=axes[0], color='lightblue', edgecolor='black', width=0.8)
axes[0].set_title("(A) Posts by Stock(s)", fontweight='bold')
axes[0].set_xlabel("Stock(s) Mentioned")
axes[0].set_ylabel("Number of Posts")
axes[0].tick_params(axis='x', rotation=45)

# Plot 2: Posts by subreddit
subreddit_counts = reddit_posts_raw['subreddit'].value_counts()
subreddit_counts.plot(kind='bar', ax=axes[1], color='lightgreen', edgecolor='black', width=0.8)
axes[1].set_title("(B) Posts by Subreddit", fontweight='bold')
axes[1].set_xlabel("Subreddit")
axes[1].set_ylabel("Number of Posts")
axes[1].tick_params(axis='x', rotation=45)

# Plot 3: Posts per month
month_counts = reddit_posts_raw['datetime'].dt.to_period('M').value_counts().sort_index()
month_counts.plot(kind='bar', ax=axes[2], color='lightcoral', edgecolor='black')
axes[2].set_title("(C) Posts by Month", fontweight='bold')
axes[2].set_xlabel("Month")
axes[2].set_ylabel("Number of Posts")
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Text-based statistics (unchanged, just remove the subreddit part)
print("--- General Statistics ---")
print(f"   Total Posts Collected: {len(reddit_posts_raw)}")
print(f"   Date Range: {reddit_posts_raw['datetime'].min().date()} to {reddit_posts_raw['datetime'].max().date()}")
print(f"   Posts with Empty Content: {reddit_posts_raw['content'].isna().sum()} ({reddit_posts_raw['content'].isna().sum()/len(reddit_posts_raw)*100:.1f}%)")
print(f"   Posts with Empty Title: {reddit_posts_raw['title'].isna().sum()} ({reddit_posts_raw['title'].isna().sum()/len(reddit_posts_raw)*100:.1f}%)")

print("\n--- Posts by Stock Analysis (A) ---")
single_stock_posts = sum(1 for stock in reddit_posts_raw['stock'] if ', ' not in stock)
multi_stock_posts = len(reddit_posts_raw) - single_stock_posts
print(f"   Single-stock posts: {single_stock_posts} ({single_stock_posts/len(reddit_posts_raw)*100:.1f}%)")
print(f"   Multi-stock posts: {multi_stock_posts} ({multi_stock_posts/len(reddit_posts_raw)*100:.1f}%)")

print("\n--- Posts by Subreddit Analysis (B) ---")
print(f"   Avg posts per subreddit: {(len(reddit_posts_raw) / len(subreddit_counts)):.2f}")
print(f"   Skewness: {skew(subreddit_counts.values):.3f}")
print(f"   Most active subreddit: r/{subreddit_counts.index[0]} ({subreddit_counts.iloc[0]} posts)")
print(f"   Least active subreddit: r/{subreddit_counts.index[-1]} ({subreddit_counts.iloc[-1]} posts)")

print("\n--- Posts by Month Analysis (C) ---")
print(f"   Avg posts per month: {(len(reddit_posts_raw) / len(month_counts)):.2f}")
print(f"   Skewness: {skew(month_counts.values):.3f}")
print(f"   Month with most posts: {month_counts.idxmax()} ({month_counts.max()} posts)")
print(f"   Month with fewest posts: {month_counts.idxmin()} ({month_counts.min()} posts)")

### Identified Data Quality Issues

1. **Multi-Stock Posts**  
   16.3% of posts (163/1003) mention multiple stocks.

2. **Subreddit Skew**  
   Post distribution across subreddits is moderately skewed, with r/stocks (265 posts) being most active and r/StocksAndTrading (93 posts) least active.

3. **Temporal Skew**  
   Post volume per month is moderately skewed, with October 2024 having only 5 posts and March 2025 having 122 posts.

4. **Content Gaps**  
   16.5% of posts lack body content, relying solely on titles for sentiment.

*These issues will be addressed in the Data Cleaning section below.*

### 4.3.3 Temporal Alignment Validation

Finally, we verify that our stock price and Reddit sentiment data cover overlapping time periods, which is essential for a valid correlation analysis.

In [ ]:
stock_data_start = stocks_data_raw.index.min().date()
stock_data_end = stocks_data_raw.index.max().date()
reddit_posts_start = reddit_posts_raw['datetime'].min().date()
reddit_posts_end = reddit_posts_raw['datetime'].max().date()

overlap_start = max(stock_data_start, reddit_posts_start)
overlap_end = min(stock_data_end, reddit_posts_end)
overlap_days = (overlap_end - overlap_start).days + 1

print("Coverage Summary")
print(f"Stock Data:      {stock_data_start} to {stock_data_end}")
print(f"Reddit Data:     {reddit_posts_start} to {reddit_posts_end}")
print(f"Overlap Period:  {overlap_start} to {overlap_end}")
print(f"Overlap Duration: {overlap_days} days ({overlap_days/7:.1f} weeks)")

print("\nAlignment Metrics")
total_possible_days = (max(stock_data_end, reddit_posts_end) - min(stock_data_start, reddit_posts_start)).days + 1
print(f"Temporal Coverage: {(overlap_days / total_possible_days) * 100:.2f}% of total period")

### Temporal Alignment Validation Assessment

- **Extensive Temporal Overlap**  
  365 days of overlapping data (99.73% coverage) provides a solid foundation for weekly analysis.

- **Compatible with Weekly Methodology**  
  The date ranges contain sufficient data to extract 50-52 complete weeks for our Friday-to-Friday analysis approach (accounting for potential incomplete first/last weeks).

- **Conclusion**  
  The raw datasets show extensive temporal overlap suitable for weekly aggregation. The exact number of complete Friday-to-Friday weeks will be confirmed during the feature engineering phase.

# 5. Data Preprocessing & Feature Engineering

## 5.1 Addressing Data Quality Issues

Based on the initial inspection, the following data quality issues were identified and will be addressed:

* **Issue 1: Multi-Stock Posts**  
  Finding: 163 posts (16.3%) mention multiple stocks, creating ambiguous sentiment attribution.  
  Solution: These posts will be duplicated for each mentioned stock to ensure clean stock-specific analysis.  
  Limitation: This approach assumes sentiment applies equally to all mentioned stocks, which may not reflect nuanced discussions where stocks are mentioned with different sentiment valences (e.g., "I love NVDA but hate TSLA"). This is a noted constraint of our sentiment aggregation methodology.

* **Issue 2: Empty Content**  
  Finding: 165 posts (16.5%) have empty content.  
  Solution: The post title will be used as the primary content for sentiment analysis, as titles typically contain the core sentiment.

* **Issue 3: Temporal Volatility in Post Volume**  
  Finding: Post volume shows moderate volatility, with particularly low volume in October 2024 (5 posts).  
  Solution: Weekly aggregation will smooth this volatility. The impact of low-volume periods on weekly sentiment averages will be considered during analysis.

## 5.2 Data Cleaning

In [ ]:
# Copy for processing
reddit_posts_clean = reddit_posts_raw.copy()
print(f"Initial dataset: {len(reddit_posts_raw)} posts")

# 1. Multi-Stock Posts
print("\n1. Processing multi-stock posts...")

expanded_posts = []
for i, row in reddit_posts_raw.iterrows():
    if ', ' in row['stock']:
        # Split stock symbols
        multiple_stocks = [s.strip() for s in row['stock'].split(', ')]
        for stock in multiple_stocks:
            new_row = row.copy()
            new_row['stock'] = stock
            new_row['was_multi_stock'] = True  # Flag for tracking
            expanded_posts.append(new_row)
    else:
        row['was_multi_stock'] = False  # Flag for tracking
        expanded_posts.append(row)

reddit_posts_clean = pd.DataFrame(expanded_posts).reset_index(drop=True)
print(f"   After expansion: {len(reddit_posts_clean)} individual stock-mentions")

# 2. Handle Empty Content
print("\n2. Handling empty content...")

reddit_posts_clean['content_clean'] = reddit_posts_clean['content'].fillna('')
reddit_posts_clean['text_for_analysis'] = reddit_posts_clean['title'] + ' ' + reddit_posts_clean['content_clean']
reddit_posts_clean['text_for_analysis'] = reddit_posts_clean['text_for_analysis'].str.strip()

# Check for completely empty text
empty_text_count = (reddit_posts_clean['text_for_analysis'].str.len() == 0).sum()
print(f"   Posts with no text after cleaning: {empty_text_count}")

print("\n=== DATA CLEANING COMPLETED ===")

### 5.2.1 Data Cleaning Validation

In [ ]:
# Create comparison figure - 3 rows, 2 columns
fig, axes = plt.subplots(3, 2, figsize=(16, 12))

# Calculate all statistics first
stock_counts_before = reddit_posts_raw['stock'].value_counts()
stock_counts_after = reddit_posts_clean['stock'].value_counts()
subreddit_counts_before = reddit_posts_raw['subreddit'].value_counts()
subreddit_counts_after = reddit_posts_clean['subreddit'].value_counts()
month_counts_before = reddit_posts_raw['datetime'].dt.to_period('M').value_counts().sort_index()
month_counts_after = reddit_posts_clean['datetime'].dt.to_period('M').value_counts().sort_index()

# Row 1: Stock Distribution
# Left: BEFORE
stock_counts_before.plot(kind='bar', ax=axes[0,0], color='lightblue', edgecolor='black', width=0.8)
axes[0,0].set_title("(A) BEFORE: Posts by Stock", fontweight='bold')
axes[0,0].set_xlabel("Stock(s)")
axes[0,0].set_ylabel("Number of Posts")
axes[0,0].tick_params(axis='x', rotation=45)

# Right: AFTER
stock_counts_after.plot(kind='bar', ax=axes[0,1], color='lightblue', edgecolor='black', width=0.8)
axes[0,1].set_title("(B) AFTER: Posts by Stock", fontweight='bold')
axes[0,1].set_xlabel("Stock")
axes[0,1].set_ylabel("Number of Posts")
axes[0,1].tick_params(axis='x', rotation=45)

# Row 2: Subreddit Distribution
# Left: BEFORE
subreddit_counts_before.plot(kind='bar', ax=axes[1,0], color='lightgreen', edgecolor='black', width=0.8)
axes[1,0].set_title("(C) BEFORE: Posts by Subreddit", fontweight='bold')
axes[1,0].set_xlabel("Subreddit")
axes[1,0].set_ylabel("Number of Posts")
axes[1,0].tick_params(axis='x', rotation=45)

# Right: AFTER
subreddit_counts_after.plot(kind='bar', ax=axes[1,1], color='lightgreen', edgecolor='black', width=0.8)
axes[1,1].set_title("(D) AFTER: Posts by Subreddit", fontweight='bold')
axes[1,1].set_xlabel("Subreddit")
axes[1,1].set_ylabel("Number of Posts")
axes[1,1].tick_params(axis='x', rotation=45)

# Row 3: Temporal Distribution
# Left: BEFORE
month_counts_before.plot(kind='bar', ax=axes[2,0], color='lightcoral', edgecolor='black')
axes[2,0].set_title("(E) BEFORE: Posts by Month", fontweight='bold')
axes[2,0].set_xlabel("Month")
axes[2,0].set_ylabel("Number of Posts")
axes[2,0].tick_params(axis='x', rotation=45)

# Right: AFTER
month_counts_after.plot(kind='bar', ax=axes[2,1], color='lightcoral', edgecolor='black')
axes[2,1].set_title("(F) AFTER: Posts by Month", fontweight='bold')
axes[2,1].set_xlabel("Month")
axes[2,1].set_ylabel("Number of Posts")
axes[2,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Distribution changes below each plot category
print("=== DISTRIBUTION CHANGES ===")

print("\n--- Stock Distribution (A/B) ---")
multi_stock_before = len([s for s in reddit_posts_raw['stock'] if ', ' in s])
print(f"   Posts:  {len(reddit_posts_clean)}")
print(f"     (BEFORE: {len(reddit_posts_raw)} posts, {multi_stock_before} multi-stock posts)")
print(f"   -> {len(reddit_posts_clean) - len(reddit_posts_raw)} posts increase")

print("\n--- Subreddit Distribution (C/D) ---")
print(f"   Avg posts per Subreddit: {(len(reddit_posts_clean) / len(subreddit_counts_after)):.2f}")
print(f"     (BEFORE {(len(reddit_posts_raw) / len(subreddit_counts_before)):.2f})")
print(f"   Skewness: {skew(subreddit_counts_after.values):.3f}")
print(f"     (BEFORE: {skew(subreddit_counts_before.values):.3f})")
print(f"   Most active: r/{subreddit_counts_after.index[0]} ({subreddit_counts_after.iloc[0]} posts)")
print(f"     (BEFORE: r/{subreddit_counts_before.index[0]} ({subreddit_counts_before.iloc[0]} posts))")
print(f"   Least active: r/{subreddit_counts_after.index[-1]} ({subreddit_counts_after.iloc[-1]} posts)")
print(f"     (BEFORE: r/{subreddit_counts_before.index[-1]} ({subreddit_counts_before.iloc[-1]} posts))")

print("\n--- Temporal Distribution (E/F) ---")
print(f"   Avg posts per Month: {(len(reddit_posts_clean) / len(month_counts_after)):.2f}")
print(f"     (BEFORE {(len(reddit_posts_raw) / len(month_counts_before)):.2f})")
print(f"   Skewness: {skew(month_counts_after.values):.3f}")
print(f"     (BEFORE: {skew(month_counts_before.values):.3f})")
print(f"   Month with most posts: {month_counts_after.idxmax()} ({month_counts_after.max()} posts)")
print(f"     (BEFORE: {month_counts_before.idxmax()} ({month_counts_before.max()} posts)")
print(f"   Month with fewest posts: {month_counts_after.idxmin()} ({month_counts_after.min()} posts)")
print(f"     (BEFORE: {month_counts_before.idxmin()} ({month_counts_before.min()} posts))")

### Data Cleaning Validation Assessment

- **Multi-Stock Issue Resolved**  
  The increase in posts (197 additional posts) confirms successful expansion of multi-stock posts. Each row now contains exactly one stock for clean analysis.

- **Data Integrity Maintained**  
  The consistent temporal pattern (same peak/valley months) and similar skewness values show that cleaning preserved the underlying data structure while resolving the multi-stock ambiguity.

- **Notable Shift in Subreddit Ranking**  
  The most active subreddit changed from r/stocks to r/investing after cleaning. This suggests r/investing had more multi-stock discussions, which is reasonable given its broader investment focus compared to r/stocks' single-stock discussions.

## 5.3 Feature Engineering

This section creates the analytical features needed for correlation analysis:

**A. Post-Level Feature Engineering**
- Add sentiment scores to each post (`textblob_score`, `vader_score`, `combined_score`)
- Add categorical `combined_sentiment_label` (positive/neutral/negative) based on the combined score 
- Add temporal feature `week_ending` (Friday-based) to each post

**B. Weekly Aggregation**  
- Calculate weekly percentage returns for each stock (Friday-to-Friday)
- Drop incomplete weeks (no prior Friday for returns / Friday outside of stock date range)
- Aggregate post-level sentiment to weekly means per stock
- Align weekly sentiment scores with weekly stock returns using Friday week-ending dates

### 5.3.1 Post-Level Feature Engineering

In [ ]:
# Initialize sentiment analyzers
vader_analyzer = SentimentIntensityAnalyzer()
print("✓ Sentiment analyzer initialized")

def analyze_sentiment(text):
    """
    Performs sentiment analysis on text using both VADER and TextBlob analyzers.
    
    Combines scores from both analyzers with equal weighting and classifies 
    sentiment as positive, negative, or neutral based on thresholds.
    
    Args:
        text (str): Input text to analyze
        
    Returns:
        dict: Dictionary containing sentiment scores and labels:
            - 'textblob_score' (float): Polarity score from TextBlob [-1.0 to 1.0]
            - 'vader_score' (float): Compound score from VADER [-1.0 to 1.0] 
            - 'combined_score' (float): Average of both scores [-1.0 to 1.0]
            - 'combined_sentiment_label' (str): 'positive', 'negative', or 'neutral'
    """
    
    text = str(text).strip()
    
    # VADER sentiment analysis
    try:
        vader_compound = vader_analyzer.polarity_scores(text)['compound']
    except Exception as e:
        print(f"VADER analysis failed for text: {text[:20]}... Error: {e}")
        vader_compound = 0.0
    
    # TextBlob sentiment analysis
    try:
        blob = TextBlob(text)
        textblob_polarity = blob.sentiment.polarity
    except Exception as e:
        print(f"TextBlob analysis failed for text: {text[:20]}... Error: {e}")
        textblob_polarity = 0.0
    
    # Combined score (equal weighting)
    combined_score = (vader_compound + textblob_polarity) / 2
    
    # Research-based threshold selection
    # Based on common NLP practice: > 0.05 for positive, < -0.05 for negative
    # Thresholds provide clear separation while avoiding over-sensitivity to noise
    if combined_score > 0.05:
        sentiment_label = 'positive'
    elif combined_score < -0.05:
        sentiment_label = 'negative'
    else:
        sentiment_label = 'neutral'
        
    return {
        'textblob_score': textblob_polarity,
        'vader_score': vader_compound,
        'combined_score': combined_score,
        'combined_sentiment_label': sentiment_label,
    }

print("\nApplying sentiment analysis...")

# Applies sentiment analysis and expands results into separate columns
sentiment_results = reddit_posts_clean['text_for_analysis'].apply(analyze_sentiment)
reddit_posts_sentiment = pd.concat([
    reddit_posts_clean, 
    pd.DataFrame(sentiment_results.tolist())
], axis=1)
#sentiment_df = pd.json_normalize(sentiment_results)
#reddit_posts_sentiment = reddit_posts_clean.assign(**sentiment_df.to_dict('series'))

# Validation
print("✓ Sentiment analysis completed\n")
print(f"   Posts with Textblob scores: {len(reddit_posts_sentiment['textblob_score'])}/{len(reddit_posts_clean)}")
print(f"   Posts with VADER scores: {len(reddit_posts_sentiment['vader_score'])}/{len(reddit_posts_clean)}")
print(f"   Posts with Combined scores: {len(reddit_posts_sentiment['combined_score'])}/{len(reddit_posts_clean)}")

# Add Temporal Feature for Weekly Analysis
print("\nAdding Friday-based week_ending to each post...")

reddit_posts_sentiment['week_ending'] = reddit_posts_sentiment['datetime'].dt.to_period('W-FRI').dt.end_time.dt.date

print(f"   Added week_ending for {reddit_posts_sentiment['week_ending'].nunique()} unique weeks")
print(f"   Date range: {reddit_posts_sentiment['datetime'].min().date()} to {reddit_posts_sentiment['datetime'].max().date()}")

### 5.3.2 Weekly Aggregation

In [ ]:
# --- Stock Returns Calculation ---
print("\n--- Weekly Stock Returns Calculation ---")

def calculate_weekly_stock_returns(stocks_df):
     """
    Calculate weekly stock returns for correlation analysis.
    
    Args:
        stocks_df (DataFrame): DataFrame with stocks data containing columns ['ticker', 'close'] and DateTimeIndex
        
    Returns:
        DataFrame: Weekly returns with columns ['ticker', 'week_ending', 'weekly_return']
    """
    weekly_returns_list = []
    
    for ticker in stocks_df['ticker'].unique():
        ticker_data = stocks_df[stocks_df['ticker'] == ticker].copy()
        
        # Weekly returns (Friday to Friday)
        weekly_data = ticker_data.resample('W-FRI').last()
        weekly_data['weekly_return'] = weekly_data['close'].pct_change()
        weekly_data['week_ending'] = weekly_data.index.date
        weekly_data['ticker'] = ticker

        # Remove first week if no prior closing (no weekly_return value)
        weekly_data = weekly_data.dropna()
        
        # Remove last week if it is incomplete (last date is not a Friday)
        last_week_ending_date = weekly_data.index[-1]
        last_actual_date = ticker_data.index[-1]
        
        if last_week_ending_date > last_actual_date:
            weekly_data = weekly_data.iloc[:-1]  # Remove last week
        
        weekly_returns_list.append(weekly_data)
        print(f"  {ticker}: {len(weekly_data)} complete weeks")
        print(f"    {weekly_data['week_ending'].min()} - {weekly_data['week_ending'].max()}")
    
    # Combine all tickers into one DataFrame
    combined_returns = pd.concat(weekly_returns_list, ignore_index=False)
    return combined_returns

# Calculate returns for all stocks
stocks_returns_weekly = calculate_weekly_stock_returns(stocks_data_raw)
print("✓ Stock returns calculated")


# --- Aggregate Sentiment to Weekly Averages ---
print("\n--- Weekly Sentiment Aggregation ---")
def aggregate_weekly_stock_sentiment(sentiment_df, returns_df):
    """
    Aggregate sentiment scores to weekly averages aligned with stock returns.
    
    Args:
        sentiment_df (DataFrame): Reddit posts with sentiment scores and week_endings
        returns_df (DataFrame): Weekly stock returns with week_endings
        
    Returns:
        DataFrame: Weekly sentiment averages with post counts
    """
    
    # Get valid stock-week combinations
    valid_combinations = returns_df[['ticker', 'week_ending']].drop_duplicates()
    valid_combinations = valid_combinations.rename(columns={'ticker': 'stock'})

    # Single merge operation for all stocks
    filtered_sentiment = sentiment_df.merge(valid_combinations, on=['stock', 'week_ending'])
    
    # Single aggregation for all stocks
    weekly_sentiment = filtered_sentiment.groupby(['stock', 'week_ending']).agg({
        'vader_score': 'mean',
        'textblob_score': 'mean', 
        'combined_score': 'mean',
        'id': 'count'
    }).reset_index()
    
    weekly_sentiment = weekly_sentiment.rename(columns={'id': 'post_count'})
    
    # Recalculate sentiment_label based on weekly combined_score mean
    weekly_sentiment['combined_sentiment_label'] = weekly_sentiment['combined_score'].apply(
        lambda score: 'positive' if score > 0.05 else 'negative' if score < -0.05 else 'neutral'
    )

    return weekly_sentiment

reddit_sentiment_weekly = aggregate_weekly_stock_sentiment(reddit_posts_sentiment, stocks_returns_weekly)
print("✓ Weekly sentiment aggregation completed")
print(f"   Total aligned weekly observations: {len(reddit_sentiment_weekly)}")
print(f"   Unique weeks: {reddit_sentiment_weekly['week_ending'].nunique()}")

## 5.4 Weekly Data Validation

Before merging, we validate both weekly datasets using these quality thresholds:

**Quality Thresholds:**
- **Reliable sentiment**: ≥3 posts per week for stable averages  
- **No data gaps**: Continuous weekly observations
- **Adequate alignment**: Sufficient overlapping weeks for correlation analysis

The validation below assesses these criteria and identifies any reliability concerns.

### 5.4.1 Weekly Stock Returns Validation
Validate stock returns data for completeness, distribution, and temporal coverage.

In [ ]:
plt.figure(figsize=(12, 4))
pivot_returns = stock_returns_weekly.pivot(index='week_ending', columns='ticker', values='weekly_return')

for ticker in pivot_returns.columns:
    plt.plot(pivot_returns.index, pivot_returns[ticker], marker='o', label=ticker, markersize=2, color=stocks_config[ticker]['color'], alpha=0.8)

plt.title("Weekly Stock Returns")
plt.xlabel("Week Ending")
plt.ylabel("Weekly Return")
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

for ticker in stock_returns_weekly['ticker'].unique():
    ticker_data = stock_returns_weekly[stock_returns_weekly['ticker'] == ticker]
    missing_returns = ticker_data['weekly_return'].isna().sum()
    print(f"{ticker}")
    print(f"  Full Weeks: {len(ticker_data)}")
    print(f"  Missing Returns: {missing_returns}")
    print(f"  Date Range: {ticker_data['week_ending'].min()} - {ticker_data['week_ending'].max()}")

### Weekly Stock Returns Validation Assessment:

- All three stocks have complete weekly returns data for 51 weeks with no gaps
- No missing returns in any stock-week combination

### 5.4.2 Weekly Reddit Sentiment Validation

Validate Reddit post coverage for gaps, low-volume weeks, and temporal coverage.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# sorted weeks for consistent x-axis
all_weeks = sorted(reddit_sentiment_weekly['week_ending'].unique())

# plot for each stock
for i, stock in enumerate(stocks_config.keys()):
    stock_data = reddit_sentiment_weekly[reddit_sentiment_weekly['stock'] == stock].sort_values('week_ending')
    
    weeks = stock_data['week_ending']
    posts = stock_data['post_count']
    
    bars = axes[i].bar(weeks, posts, color='lightblue', edgecolor='black', width=5)
    
    # Highlights low-volume weeks (<3 posts) in red
    for j, (week, count) in enumerate(zip(weeks, posts)):
        if count < 3:
            bars[j].set_color('red')
    
    axes[i].set_title(f"{stock} - Weekly Post Volume", fontweight='bold')
    axes[i].set_ylabel("Number of Posts")
    axes[i].axhline(y=3, color='red', linestyle='--', alpha=0.7, label="Minimum threshold (3 posts)")
    axes[i].legend()
    axes[i].tick_params(axis='x', rotation=45)
    
    # Only show x-axis labels on bottom plot to reduce clutter
    if i < 2:
        axes[i].set_xticklabels([])
    else:
        axes[i].set_xlabel("Week Ending Date")

plt.tight_layout()
plt.show()

# Add summary statistics
print("\n--- Weekly Post Volume Summary ---")
for stock in stocks_config.keys():
    stock_data = reddit_sentiment_weekly[reddit_sentiment_weekly['stock'] == stock]
    low_volume = len(stock_data[stock_data['post_count'] < 3])
    zero_posts = len(stock_data[stock_data['post_count'] == 0])
    
    print(f"{stock}:")
    print(f"  Total weeks: {len(stock_data)}")
    print(f"  Total number of posts: {stock_data['post_count'].sum()}")
    print(f"  Weeks with <3 posts: {low_volume} ({low_volume/len(stock_data)*100:.2f}%)")
    print(f"  Weeks with 0 posts: {zero_posts}")
    print(f"  Average posts/week: {stock_data['post_count'].mean():.1f}")

print("\nGeneral:")
print(f"  Total weeks: {len(reddit_sentiment_weekly)}")
print(f"  Total number of posts: {reddit_sentiment_weekly['post_count'].sum()}")
low_volume_all = len(reddit_sentiment_weekly[reddit_sentiment_weekly['post_count'] < 3])
zero_posts_all = len(reddit_sentiment_weekly[reddit_sentiment_weekly['post_count'] == 0])
print(f"  Weeks with <3 posts: {low_volume_all} ({low_volume_all/len(reddit_sentiment_weekly)*100:.2f}%)")
print(f"  Weeks with 0 posts: {zero_posts_all}")

### Weekly Reddit Sentiment Validation Assessment

**Reliability**  
- **NVDA**: Excellent coverage with only 3.92% of weeks having low post volume
- **TSLA**: Good coverage though with higher low-volume weeks (9.80%) - sentiment may be noisier
- **GOOGL**: Solid coverage with 7.84% low-volume weeks

**Quality**  
- 92.81% of weeks (142/153) have adequate post volume (≥3 posts)
- Consistent 51-week coverage across all stocks
- No gaps in temporal coverage

### 5.4.3 Weekly Alignment Validation

Verify that the weekly aggregated sentiment data and stock returns are correctly aligned by date and stock identifier to ensure valid correlation analysis.

In [ ]:
for ticker in stocks_config.keys():
    # Returns data for this stock
    returns_data = stock_returns_weekly[stock_returns_weekly['ticker'] == ticker]
    returns_weeks = set(returns_data['week_ending'])
    
    # Sentiment data for this stock
    sentiment_data = reddit_sentiment_weekly[reddit_sentiment_weekly['stock'] == ticker]
    sentiment_weeks = set(sentiment_data['week_ending'])
    
    # Calculate alignment
    overlapping_weeks = returns_weeks & sentiment_weeks
    total_possible_weeks = len(returns_weeks | sentiment_weeks)
    alignment_percentage = (len(overlapping_weeks) / total_possible_weeks) * 100
    
    print(f"\n{ticker}:")
    print(f"  Weekly Returns:    {len(returns_weeks)} weeks, {returns_data['week_ending'].min()} to {returns_data['week_ending'].max()}")
    print(f"  Weekly Sentiment:  {len(sentiment_weeks)} weeks, {sentiment_data['week_ending'].min()} to {sentiment_data['week_ending'].max()}")
    print(f"  Overlap:           {len(overlapping_weeks)} weeks ({alignment_percentage:.1f}% alignment)")
    
    # Show any mismatches
    if len(overlapping_weeks) < len(returns_weeks):
        missing_in_sentiment = returns_weeks - sentiment_weeks
        missing_in_returns = sentiment_weeks - returns_weeks
        if missing_in_sentiment:
            print(f"    Missing in sentiment: {len(missing_in_sentiment)} weeks")
        if missing_in_returns:
            print(f"    Missing in returns: {len(missing_in_returns)} weeks")

### Weekly Alignment Validation Assessment

**Temporal Coverage**  
- All three stocks have weekly returns for 51 weeks with no gaps.
   - (2024-11-08 - 2025-10-24)
- All three stocks have sentiment data for 51 weeks with no gaps.
   - (2024-11-08 - 2025-10-24)
    
**Alignment Quality**  
With 92.81% of weeks having adequate post volume (≥3 posts) and consistent 51-week coverage, the dataset provides a robust foundation for correlation analysis.

**Methodological Decision**  
All 51 weeks will be included in the correlation analysis. While weeks with fewer than 3 posts may have noisier sentiment measurements, they represent genuine periods of lower market discussion and provide a complete picture of the relationship over time.

## 5.5 Dataset Merging & Final Validation

This section combines the engineered weekly features into the final analysis dataset:

- **Merge** weekly sentiment scores with weekly stock returns
- **Validate** the merged dataset structure and completeness

### 5.5.1 Dataset Merging

With both datasets validated and temporally aligned, we now merge the weekly sentiment averages with weekly stock returns to create the final analysis dataset. The merge uses an inner join on `stock` and `week_ending` columns to ensure only complete stock-week pairs are included.

In [ ]:
# Prepare returns data for merging
returns_for_merge = stocks_returns_weekly[['ticker', 'week_ending', 'weekly_return']].copy()
returns_for_merge = returns_for_merge.rename(columns={'ticker': 'stock'})

print(f"Returns data for merging: {len(returns_for_merge)} observations")
print(f"Sentiment data for merging: {len(reddit_sentiment_weekly)} observations")

# Merge the datasets
merged_data = reddit_sentiment_weekly.merge(
    returns_for_merge,
    on=['stock', 'week_ending'],
    how='inner'  # Should be perfect match due to previous alignment
)

print(f"✓ Merged dataset created: {len(merged_data)} observations")
print(f"  Stocks: {merged_data['stock'].unique().tolist()}")
print(f"  Weeks: {merged_data['week_ending'].nunique()}")
print(f"  Date range: {merged_data['week_ending'].min()} to {merged_data['week_ending'].max()}")

### 5.5.2 Merged Dataset Validation

After merging, we perform a final validation to ensure the dataset is analysis-ready.  
This checks data integrity, completeness, and confirms the merged dataset meets all quality standards for correlation analysis.

In [ ]:
print("--- 1. Dataset Structure ---")
print(f"   Rows: {len(merged_data)}")
print(f"   Columns: {len(merged_data.columns)}")
print(f"   Shape: {merged_data.shape}")

print("\n--- 2. Data Completeness ---")
print(f"   Missing values:")
for col in merged_data.columns:
    missing = merged_data[col].isna().sum()
    if missing > 0:
        print(f"     ✗ {col}: {missing} missing")
    else:
        print(f"     ✓ {col}: complete")

print("\n--- 3. Data Quality ---")
print(f"   Expected observations: {merged_data['stock'].nunique()} stocks × {merged_data['week_ending'].nunique()} weeks = {merged_data['stock'].nunique() * merged_data['week_ending'].nunique()}")
print(f"     Actual observations: {len(merged_data)}")
duplicates = merged_data.duplicated(subset=['stock', 'week_ending']).sum()
print(f"   Duplicate stock-week pairs: {duplicates}")

### Merged Dataset Validation Assessment

- Perfect structural alignment: 153 rows matching the expected 3 stocks × 51 weeks
- No duplicate stock-week pairs or missing values across all 8 columns
- Complete temporal coverage with all weeks represented for each stock

The merged dataset successfully integrates all engineered features and meets all quality standards for correlation analysis.

# 6. Exploratory Data Analysis

Before testing our hypothesis about sentiment and returns, we first examine the fundamental characteristics of our key variables. This initial exploration ensures we understand what we're working with and helps identify any data issues that might affect our correlation analysis.

## 6.1 Distribution of Target Variable: Weekly Returns

We begin by analyzing the weekly returns for NVDA, TSLA, and GOOGL — the core metric we're trying to relate to Reddit sentiment. This examination answers basic but crucial questions: How volatile are these stocks? What are their typical weekly movements? Are there extreme values that could distort our results?

The findings here establish the baseline behavior of our dependent variable and inform whether standard correlation methods are appropriate for this dataset.

In [ ]:
# subplots for histograms and boxplots
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

for i, ticker in enumerate(stocks_config.keys()):
    stock_data = merged_data[merged_data['stock'] == ticker]
    returns = stock_data['weekly_return']
    
    # Histogram (top row)
    axes[i].hist(returns, bins=15, color=stocks_config[ticker]['color'], alpha=0.6, edgecolor='black')
    axes[i].axvline(returns.mean(), color='red', linestyle='--', linewidth=2, label=f"Mean: {returns.mean():.3f}")
    axes[i].set_title(f"{ticker} - Weekly Returns Distribution")
    axes[i].set_xlabel("Weekly Return")
    axes[i].set_ylabel("Frequency")
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n--- Distribution Summary ---")
for ticker in stocks_config.keys():
    stock_data = merged_data[merged_data['stock'] == ticker]
    returns = stock_data['weekly_return']
    
    print(f"\n{ticker}:")
    print(f"  Mean: {returns.mean() * 100:.2f}%")
    print(f"  Std Dev: {returns.std() * 100:.2f}%")
    print(f"  Min: {returns.min() * 100:.2f}%")
    print(f"  Max: {returns.max() * 100:.2f}%")
    print(f"  Skewness: {skew(returns):.4f}")

### Insights from Weekly Returns Distribution

The descriptive statistics reveal distinct risk-return profiles across the three technology stocks during the analysis period:

**Volatility Hierarchy**
- **TSLA** exhibits the highest volatility (Std Dev: 8.21%), confirming its reputation as the most unpredictable of the three
- **NVDA** shows moderate volatility (Std Dev: 6.44%) 
- **GOOGL** demonstrates the most stable behavior (Std Dev: 4.42%), approximately half as volatile as TSLA

**Return Characteristics**
- All three stocks achieved positive average weekly returns, consistent with the overall bullish market context
- TSLA delivered the highest average returns (1.41%) but with substantially greater risk
- GOOGL offered the most favorable risk-return profile with steady, positive performance

**Distribution Patterns:**
- **TSLA** shows significant positive skewness (0.8109), showing a tendency for occasional large positive returns that pull the average upward
- **GOOGL** displays a nearly symmetric distribution (skewness: 0.0014), while **NVDA** shows very slight negative skewness (-0.0149), showing more balanced upside and downside movements
- The maximum weekly gains exceed maximum losses for all stocks, particularly for TSLA which saw a 29.01% weekly surge

#### Implications for Sentiment Analysis
The varying volatility levels provide an ideal testing ground for our hypothesis.
If Reddit sentiment correlates with returns, we would expect to see the strongest relationship with TSLA (highest volatility) and the weakest with GOOGL (lowest volatility).

## 6.2 Distribution of Predictor Variables: Sentiment Scores

We now examine the sentiment scores that form the basis of our predictive analysis. Understanding the distribution of VADER, TextBlob, and combined sentiment scores is crucial for assessing whether these variables contain meaningful signal for our correlation analysis.

This exploration answers fundamental questions: Do we have sufficient variation in sentiment scores to detect relationships with returns? Are the scores systematically biased in one direction? How consistent are the different sentiment measurement methods?

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(15, 12))

sentiment_columns = ['vader_score', 'textblob_score', 'combined_score']
sentiment_names = ['VADER Sentiment', 'TextBlob Sentiment', 'Combined Sentiment']

for i, ticker in enumerate(stocks_config.keys()):
    stock_data = merged_data[merged_data['stock'] == ticker]
    
    for j, (col, name) in enumerate(zip(sentiment_columns, sentiment_names)):
        sentiment_data = stock_data[col]
        
        # Histograms
        axes[j, i].hist(sentiment_data, bins=15, color=stocks_config[ticker]['color'], alpha=0.6, edgecolor='black')
        axes[j, i].axvline(sentiment_data.mean(), color='red', linestyle='--', linewidth=2, 
                          label=f"Mean: {sentiment_data.mean():.3f}")
        axes[j, i].axvline(0, color='black', linestyle='-', alpha=0.5, linewidth=1)
        
        if i == 0:
            axes[j, i].set_ylabel(f"{name}\nFrequency")
        if j == 0:
            axes[j, i].set_title(f"{ticker}")
        if j == 2:
            axes[j, i].set_xlabel("Sentiment Score")
            
        axes[j, i].legend()
        axes[j, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print sentiment statistics
print("\n--- Sentiment Summary ---")
for ticker in stocks_config.keys():
    stock_data = merged_data[merged_data['stock'] == ticker]
    
    print(f"\n{ticker}:")
    for col, name in zip(sentiment_columns, sentiment_names):
        sentiment_data = stock_data[col]
        mean_str = f"Mean: {sentiment_data.mean():.3f}"
        std_dev_str = f"Std Dev: {sentiment_data.std():.3f}"
        min_str = f"Min: {sentiment_data.min():.3f}"
        max_str = f"Max: {sentiment_data.max():.3f}"
        skew_str = f"Skewness: {skew(sentiment_data):.3f}"
        pos_str = f">0: {(sentiment_data > 0).sum()}/51 weeks ({(sentiment_data > 0).sum()/51*100:.2f}%)"
        print(f"  --{name}--")
        print(f"    {mean_str:12} | {std_dev_str:12} | {min_str:12} | {max_str:12} | {skew_str:18}")
        print(f"    | {pos_str}")

### Insights from Sentiment Scores Distribution

The sentiment analysis reveals striking patterns in Reddit discussion sentiment across the three technology stocks:

**Overwhelming Optimism Bias**
- **GOOGL** generated the most consistently positive sentiment, with 94.12-98.04% of weeks showing positive scores across all measurement methods
- **NVDA** follows with 84.31-92.16% positive weeks, demonstrating strong but slightly more varied sentiment
- **TSLA** showed the most balanced sentiment profile, though still predominantly positive (76.47-82.35% of weeks)

**Methodological Divergence**
- **VADER** captured much stronger sentiment signals (means: 0.24-0.52) with greater variability, reflecting its sensitivity to social media language patterns
- **TextBlob** produced more conservative, tightly clustered scores (means: 0.06-0.10), suggesting it may be less responsive to the extremes of financial discussion
- The **combined scores** effectively averaged these approaches, providing moderate sentiment values with reduced volatility

**Sentiment Strength Hierarchy**
- **GOOGL** attracted the most favorable sentiment overall, with combined scores averaging 0.31
- **NVDA** followed closely with 0.24 average combined sentiment
- **TSLA** generated the most tempered positive sentiment at 0.15, potentially reflecting its controversial nature among investors

**Distribution Characteristics**
- All three stocks show negative skewness in VADER and combined scores, indicating occasional strongly negative sentiment weeks punctuate generally positive discussion
- TextBlob distributions are generally more symmetric, though GOOGL shows notable positive skewness (0.810)
- The substantial standard deviations confirm meaningful week-to-week sentiment variation exists for correlation analysis

#### Critical Implication for Hypothesis Testing
The strong positive sentiment bias across all stocks creates a challenging environment for detecting correlation with returns. With sentiment predominantly positive, we are essentially testing whether *degrees of optimism* rather than sentiment direction correlates with price movements.

## 6.3 Temporal Patterns Analysis

We now examine how post volume, sentiment, and returns evolve over time to identify potential trends, seasonal patterns, or structural breaks that could influence our correlation analysis. Understanding the temporal dynamics is crucial for recognizing whether external events or systematic patterns might confound the relationship between Reddit sentiment and stock returns.

This analysis helps answer critical questions: Are there periods of unusually high market discussion that coincide with major price movements? Do sentiment and returns show coordinated shifts over time that might suggest broader market regimes? Are there specific events or time periods that disproportionately drive any observed relationships?

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(18, 12))

for i, ticker in enumerate(stocks_config.keys()):
    stock_data = merged_data[merged_data['stock'] == ticker].sort_values('week_ending')
    bar_color = stocks_config[ticker]['color']
    
    # Plot 1: Post Volume over Time
    axes[0, i].bar(stock_data['week_ending'], stock_data['post_count'], 
                   color=bar_color, alpha=0.6, width=5)
    axes[0, i].set_title(f"{ticker} - Weekly Post Volume")
    axes[0, i].set_ylabel("Number of Posts")
    axes[0, i].tick_params(axis='x', rotation=45)
    axes[0, i].grid(True, alpha=0.3)
    
    # Plot 2: Sentiment over Time
    axes[1, i].plot(stock_data['week_ending'], stock_data['combined_score'], 
                    color=bar_color, alpha=0.6, linewidth=2.5, marker='o', markersize=3, label="Combined Sentiment")
    axes[1, i].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[1, i].set_title(f"{ticker} - Weekly Sentiment Trend")
    axes[1, i].set_ylabel("Sentiment Score")
    axes[1, i].tick_params(axis='x', rotation=45)
    axes[1, i].grid(True, alpha=0.3)
    axes[1, i].legend()
    
    # Plot 3: Returns over Time
    axes[2, i].plot(stock_data['week_ending'], stock_data['weekly_return'], 
                    color=bar_color, alpha=0.6, linewidth=2.5, marker='o', markersize=3, label="Weekly Returns")
    axes[2, i].axhline(y=0, color='black', linestyle='--', alpha=0.5)
    axes[2, i].set_title(f"{ticker} - Weekly Returns Trend")
    axes[2, i].set_ylabel("Weekly Return")
    axes[2, i].set_xlabel("Week Ending")
    axes[2, i].tick_params(axis='x', rotation=45)
    axes[2, i].grid(True, alpha=0.3)
    axes[2, i].legend()

plt.tight_layout()
plt.show()

# Calculate temporal statistics
print("\n--- Temporal Summary ---")
for ticker in stocks_config.keys():
    stock_data = merged_data[merged_data['stock'] == ticker].sort_values('week_ending')
    
    print(f"\n{ticker}:")
    
    # Post volume trends
    early_posts, mid_posts, late_posts = np.array_split(stock_data['post_count'], 3)
    print(f"  Post Volume Trend: Early {early_posts.mean():.2f} → Mid {mid_posts.mean():.2f} → Late {late_posts.mean():.2f}")
    
    # Sentiment trends
    early_sentiment, mid_sentiment, late_sentiment = np.array_split(stock_data['combined_score'], 3)
    print(f"  Sentiment Trend: Early {early_sentiment.mean():.3f} → Mid {mid_sentiment.mean():.3f} → Late {late_sentiment.mean():.3f}")
    
    # Volatility clusters
    high_vol_periods = len(stock_data[abs(stock_data['weekly_return']) > stock_data['weekly_return'].std()])
    print(f"  High Volatility Weeks: {high_vol_periods}/51 ({high_vol_periods/51*100:.2f}%)")

### Insights from Temporal Patterns Analysis

The temporal analysis reveals distinct patterns in how discussion volume, sentiment, and returns evolved throughout the study period:

**Discussion Volume Dynamics**
- **TSLA** experienced the most dramatic volume fluctuations, with a mid-period surge to 12.9 posts/week followed by a sharp decline, suggesting episodic bursts of intense market discussion
- **NVDA** and **GOOGL** showed more stable discussion patterns, while both see increased attention in the latter period

**Sentiment Trajectories**
- All three stocks exhibited a **mid-period sentiment dip**, with TSLA experiencing the most pronounced decline to near-neutral levels (0.072)
- **GOOGL** maintained the highest sentiment throughout, though it still followed the broader pattern of mid-period cooling
- The parallel sentiment declines across all stocks suggest **market-wide factors** rather than company-specific news drove sentiment patterns

**Volatility and Discussion Patterns**
- **GOOGL** showed the highest proportion of high-volatility weeks (33.3%), challenging its perception as the most stable stock
- The timing of TSLA's volume surge aligns with its mid-period sentiment decline, potentially indicating contentious or uncertain discussion phases
- NVDA's late-period volume increase coincides with sentiment recovery, suggesting renewed optimistic discussion

#### Critical Implications for Correlation Analysis
The synchronized mid-period sentiment decline across all stocks represents a potential confounding factor. If this sentiment dip coincided with broader market conditions that also affected returns, it could create a spurious correlation. The episodic nature of discussion volume, particularly for TSLA, means sentiment measurements may be less reliable during low-engagement periods.

The temporal patterns suggest we should test whether the sentiment-return relationship remains consistent across different phases of our study period, particularly comparing the sentiment-dip period against the more optimistic early and late phases.

# 7. Relationship Analysis

We now directly test our core hypothesis by examining the quantitative relationship between Reddit sentiment and stock returns. This section moves from exploratory analysis to hypothesis testing, using statistical methods to measure whether the anticipated connection between online discussion sentiment and price movements actually exists in our data.

The correlation analysis provides the foundational evidence for answering our research question: Is there a statistically significant relationship between weekly sentiment scores and weekly stock returns across our three technology stocks?

## 7.1 Correlation Analysis (Quantitative Measurement)

This analysis quantifies the linear relationship between Reddit sentiment scores and weekly stock returns using Pearson correlation coefficients. For each stock and sentiment measurement method, we calculate the correlation strength (r), statistical significance (p-value), and 95% confidence intervals to assess both the magnitude and precision of any observed relationships.

The R-squared values provide crucial context by indicating what percentage of return variation is explained by sentiment scores, while confidence intervals help determine whether observed correlations are distinguishable from random chance. Together, these metrics allow us to evaluate both the statistical and practical significance of any relationships found.

In [ ]:
# HELPER
sentiment_names = {
    'Vader': 'vader_score',
    'TextBlob': 'textblob_score',
    'Combined': 'combined_score'
}

def correlation_ci(r, n, confidence=0.95):
    """Calculate confidence interval for Pearson correlation coefficient using Fisher z-transform"""
    # Fisher z-transform
    z = np.arctanh(r)
    
    # Standard error
    se = 1 / np.sqrt(n - 3)
    
    # Z critical value
    z_crit = norm.ppf(1 - (1 - confidence) / 2)
    
    # Confidence interval in z-space
    z_lower = z - z_crit * se
    z_upper = z + z_crit * se
    
    # Transform back to r
    r_lower = np.tanh(z_lower)
    r_upper = np.tanh(z_upper)
    
    return (r_lower, r_upper)

# Calculate comprehensive correlation matrix
correlation_results = []

for ticker in stocks_config.keys():
    stock_data = merged_data[merged_data['stock'] == ticker]
    n_weeks = len(stock_data)
    
    for sentiment_name, sentiment_col in sentiment_names.items():
        corr_coef, p_value = pearsonr(stock_data[sentiment_col], stock_data['weekly_return'])
        
        # Calculate confidence interval
        ci_lower, ci_upper = correlation_ci(corr_coef, n_weeks)
        
        correlation_results.append({
            'stock': ticker,
            'sentiment_method': sentiment_name,
            'correlation': corr_coef,
            'p_value': p_value,
            'significant': p_value < 0.05,
            'r_squared': corr_coef ** 2,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'ci_width': ci_upper - ci_lower
        })

# Results dataframe
corr_df = pd.DataFrame(correlation_results)

print("\n--- Correlation Results ---")
for ticker in stocks_config.keys():
    print(f"\n{ticker}:")
    stock_corr = corr_df[corr_df['stock'] == ticker]
    for _, row in stock_corr.iterrows():
        significance = "***" if row['p_value'] < 0.05 else ""
        print(f"  {row['sentiment_method']:12} r = {row['correlation']:6.3f}{significance}")
        print(f"              95% CI: [{row['ci_lower']:6.3f}, {row['ci_upper']:6.3f}]")
        print(f"              p = {row['p_value']:5.3f}, R² = {row['r_squared']:5.3f}")

print("\n--- Correlation Heatmap ---")
pivot_corr = corr_df.pivot(index="sentiment_method", columns='stock', values='correlation')
pivot_pvals = corr_df.pivot(index="sentiment_method", columns='stock', values='p_value')

plt.figure(figsize=(10, 4))
sns.heatmap(pivot_corr, annot=True, cmap='RdYlBu', center=0, 
            vmin=-0.5, vmax=0.5, fmt='.3f', 
            cbar_kws={'label': "Correlation Coefficient"})
plt.title("Correlation: Sentiment Scores vs Weekly Returns")
plt.tight_layout()
plt.show()

print("\n--- Correlation Summary ---")
print(f"Total significant correlations: {corr_df['significant'].sum()}/9 ({corr_df['significant'].sum()/9*100:.1f}%)")
print(f"Average correlation coefficient: {corr_df['correlation'].mean():.3f}")
print(f"Average R²: {corr_df['r_squared'].mean():.3f}")
print(f"Average 95% CI width: {corr_df['ci_width'].mean():.3f}")

print("\n--- Method Comparison ---")
method_summary = corr_df.groupby('sentiment_method').agg({
    'correlation': 'mean',
    'significant': 'sum',
    'r_squared': 'mean',
    'ci_width': 'mean'
}).round(3)
print(method_summary)

### Insights from Correlation Analysis

The correlation results reveal a clear and definitive pattern: **no statistically significant relationship exists** between Reddit sentiment scores and weekly stock returns for any of the three technology stocks during our study period.

#### Key Findings

- **Statistical Significance**
   - **0 out of 9 correlations** reached statistical significance (p < 0.05)
   - All p-values range from 0.386 to 0.889, far above the conventional significance threshold
   - **All 95% confidence intervals include zero**, confirming the correlations are indistinguishable from random chance

- **Correlation Strength and Precision**
   - Correlation coefficients are exceptionally weak, ranging from -0.077 to +0.124
   - The average correlation across all stocks and methods is only 0.052
   - **Wide confidence intervals** (average width: 0.548) indicate substantial uncertainty in all estimates
   - Even the most extreme plausible values (CI bounds) show only modest potential relationships

- **Explanatory Power**
   - R-squared values are negligible, ranging from 0.000 to 0.015
   - On average, sentiment scores explain only **0.6%** of the variation in weekly returns
   - Even the strongest relationship (TSLA VADER) explains just 1.5% of return variation

- **Method Consistency**
   - All three sentiment measurement methods show similarly weak results and uncertainty
   - No method demonstrates clear superiority in detecting relationships with returns
   - Identical confidence interval widths (0.548-0.549) across methods confirm consistent precision

#### Critical Implications

**These results directly reject our initial hypothesis.** We hypothesized that "there is a statistically significant positive correlation between weekly Reddit sentiment scores and weekly stock returns," but the evidence shows:

1. **No statistical significance** - All correlations fail significance tests and confidence intervals include zero
2. **Substantial uncertainty** - Wide confidence intervals show we cannot precisely estimate any true relationship
3. **Minimal explanatory power** - Even if relationships exist, they explain virtually none of the return variation
4. **Methodological consistency** - The null finding holds across all sentiment measurement approaches

While the average correlation (0.052) is technically positive as hypothesized, the combination of statistical insignificance, wide confidence intervals, and trivial explanatory power means we cannot claim support for the proposed relationship. The findings strongly suggest that weekly aggregated Reddit sentiment, as measured in this study, does not correlate with short-term stock price movements.

## 7.2 Visual Relationship Analysis (Qualitative Assessment)

While correlation coefficients provide numerical evidence, visual analysis allows us to qualitatively assess the relationship patterns. This section complements our statistical findings by examining scatterplots with regression lines and confidence intervals, helping us understand whether the weak correlation results reflect genuinely scattered data or are influenced by outliers or non-linear patterns.

The visual assessment answers a crucial question: Do the scatterplots confirm the absence of relationship suggested by our correlation analysis, or do they reveal patterns that might be masked by the linear correlation assumptions?

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(15, 12))

for i, ticker in enumerate(stocks_config.keys()):
    stock_data = merged_data[merged_data['stock'] == ticker]
    
    for j, (sentiment_name, sentiment_col) in enumerate(sentiment_names.items()):
        # Scatterplot with regression line and confidence interval
        sns.regplot(
            x=sentiment_col, 
            y="weekly_return", 
            data=stock_data,
            ax=axes[i, j],
            scatter_kws={'alpha': 0.6, 'color': stocks_config[ticker]['color'], 's': 50},
            line_kws={'color': 'red', 'linewidth': 2},
            ci=95  # 95% confidence interval
        )
        
        corr = stock_data[sentiment_col].corr(stock_data['weekly_return'])
        
        axes[i, j].axhline(y=0, color='black', linestyle='--', alpha=0.3)
        axes[i, j].axvline(x=0, color='black', linestyle='--', alpha=0.3)
        axes[i, j].set_xlabel(f"{sentiment_name}")
        if j == 0:
            axes[i, j].set_ylabel("Weekly Return")
        axes[i, j].set_title(f"{ticker}\nr = {corr:.3f}")
        axes[i, j].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n--- Visual Pattern Analysis ---")
for i, ticker in enumerate(stocks_config.keys()):
    stock_data = merged_data[merged_data['stock'] == ticker]
    
    print(f"\n{ticker}")
    
    # Check for outlier influence
    for sentiment_name, sentiment_col in sentiment_names.items():
        sentiment_data = stock_data[sentiment_col]
        returns_data = stock_data['weekly_return']
        
        # Extreme points
        extreme_sentiment = (abs(sentiment_data - sentiment_data.mean()) > 2 * sentiment_data.std())
        extreme_returns = (abs(returns_data - returns_data.mean()) > 2 * returns_data.std())
        extreme_points = sum(extreme_sentiment & extreme_returns)
        
        print(f"  {sentiment_name:12}: {extreme_points} extreme points in both dimensions")
    
    # Check data distribution across quadrants
    combined_data = stock_data['combined_score']
    returns_data = stock_data['weekly_return']
    
    q1 = sum((combined_data > 0) & (returns_data > 0))  # Positive sentiment, positive returns
    q2 = sum((combined_data < 0) & (returns_data > 0))  # Negative sentiment, positive returns  
    q3 = sum((combined_data < 0) & (returns_data < 0))  # Negative sentiment, negative returns
    q4 = sum((combined_data > 0) & (returns_data < 0))  # Positive sentiment, negative returns
    
    print(f"  Quadrant Distribution: ++:{q1}/51, -+:{q2}/51, --:{q3}/51, +-:{q4}/51")

### Insights from Visual Relationship Analysis

The scatterplots and pattern analysis provide compelling visual confirmation of the weak correlation results, with consistent patterns across all three sentiment measurement methods.

#### Data Distribution Patterns (Based on Combined Sentiment Scores)

- **GOOGL** shows the most concentrated clustering:
   - **32 of 51 weeks** (63%) fall in the "positive sentiment, positive returns" quadrant
   - Only **1 week** shows the counter-intuitive "negative sentiment, positive returns" pattern
   - **Zero weeks** with aligned negative sentiment and negative returns
   - This creates a dense cluster in the upper-right quadrant with minimal spread

- **NVDA** displays moderate dispersion:
   - **29 of 51 weeks** (57%) in the positive-positive quadrant  
   - Balanced distribution across other quadrants, though still sparse
   - Shows slightly more variation than GOOGL but without clear directional pattern

- **TSLA** exhibits the most scattered distribution:
   - Only **18 of 51 weeks** (35%) in the positive-positive quadrant
   - Substantial presence in all quadrants, including **22 weeks** of "positive sentiment, negative returns"
   - This widespread scattering explains TSLA's slightly higher (but still non-significant) correlation coefficients

**Critical Consistency Across Methods**
- **No extreme outliers** were found in any of the three sentiment methods (VADER, TextBlob, Combined)
- The visual patterns in scatterplots are remarkably consistent across all sentiment measurement approaches
- This methodological consistency strengthens our confidence that the absence of relationship is genuine, not an artifact of one particular sentiment analysis technique

#### Overall Conclusion
The visual evidence strongly supports the correlation results across all sentiment measurement methods. The scatterplots show genuinely scattered data without clear linear trends, confirming that no meaningful relationship exists between weekly sentiment scores and returns in our dataset.

## 7.3 Alignment Analysis (Categorical Relationship)

This analysis provides a simplified, categorical perspective on the sentiment-return relationship by measuring how often sentiment direction aligns with return direction. Instead of measuring correlation strength, we calculate the simple percentage of weeks where sentiment and returns point in the same direction.

This approach answers a more fundamental question: If we used Reddit sentiment as a simple directional indicator, how often would it correctly match the actual movement of stock prices?

**Methodological Note**  
We compare alignment rates against expected random chance, which accounts for the baseline probability of alignment given the individual distributions of positive sentiment and positive returns. The expected random alignment is calculated as: 
`P(Positive Sentiment) × P(Positive Returns) + P(Negative Sentiment) × P(Negative Returns)`. This ensures we don't misinterpret high alignment rates that simply reflect both series being predominantly positive

In [ ]:
alignment_results = []

for ticker in stocks_config.keys():
    stock_data = merged_data[merged_data['stock'] == ticker]
    
    for sentiment_name, sentiment_col in sentiment_names.items():
        # Sentiment direction (positive/negative)
        sentiment_positive = stock_data[sentiment_col] > 0
        returns_positive = stock_data['weekly_return'] > 0
        
        # Calculate alignment
        aligned_weeks = (sentiment_positive & returns_positive) | (~sentiment_positive & ~returns_positive)
        alignment_percentage = aligned_weeks.mean() * 100
        
        # Calculate expected random alignment
        p_positive_sentiment = sentiment_positive.mean()
        p_positive_returns = returns_positive.mean()
        expected_random = (p_positive_sentiment * p_positive_returns + 
                          (1 - p_positive_sentiment) * (1 - p_positive_returns)) * 100
        
        alignment_results.append({
            'stock': ticker,
            'method': sentiment_name,
            'alignment_percentage': alignment_percentage,
            'expected_random': expected_random,
            'above_random': alignment_percentage > expected_random,
            'weeks_aligned': aligned_weeks.sum(),
            'total_weeks': len(aligned_weeks)
        })

alignment_df = pd.DataFrame(alignment_results)

print("\n--- Alignment Results ---")
for ticker in stocks_config.keys():
    print(f"\n{ticker}:")
    stock_alignment = alignment_df[alignment_df['stock'] == ticker]
    
    for _, row in stock_alignment.iterrows():
        diff_from_random = row['alignment_percentage'] - row['expected_random']
        diff_symbol = "+" if diff_from_random > 0 else ""
        print(f"  {row['method']:12}: {row['alignment_percentage']:5.2f}% aligned "
              f"({row['weeks_aligned']}/{row['total_weeks']} weeks)")
        print(f"               Expected by random chance: {row['expected_random']:5.2f}% "
              f"({diff_symbol}{diff_from_random:4.2f}%)")

# Overall summary
print("\n--- Alignment Summary ---")
overall_alignment = alignment_df['alignment_percentage'].mean()
overall_expected = alignment_df['expected_random'].mean()

print(f"Overall Alignment: {overall_alignment:.2f}%")
print(f"Overall Expected by Chance: {overall_expected:.2f}%")
print(f"Average Above Random: {overall_alignment - overall_expected:+.2f}%")
print(f"Methods Above Random: {alignment_df['above_random'].sum()}/9 "
      f"({alignment_df['above_random'].sum()/9*100:.1f}%)")

# Create visualization
fig, ax = plt.subplots(figsize=(12, 6))
x_pos = np.arange(len(alignment_df))

colors = [stocks_config[ticker]['color'] for ticker in alignment_df['stock']]

bars = ax.bar(x_pos, alignment_df['alignment_percentage'], color=colors, alpha=0.6, width=0.8, label="Actual Alignment")

# Reference lines
ax.axhline(y=50, color='red', linestyle='--', linewidth=2, label="Pure Random (50%)")

# Expected random lines
for i, (_, row) in enumerate(alignment_df.iterrows()):
    ax.plot([i-0.4, i+0.4], [row['expected_random'], row['expected_random']], 
            'black', linewidth=2, alpha=0.8, label="Expected Random" if i == 0 else "")

# Customize plot
ax.set_xlabel("Stock & Method")
ax.set_ylabel("Alignment Percentage (%)")
ax.set_title("Sentiment-Return Directional Alignment vs Random Chance")
ax.set_xticks(x_pos)
ax.set_xticklabels([f"{row['stock']}\n{row['method']}" for _, row in alignment_df.iterrows()], rotation=45)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

### Insights from Alignment Analysis

The alignment analysis **reinforces the consistent pattern** observed throughout our analysis: Reddit sentiment provides no meaningful directional signal for stock returns, with alignment rates that match or even underperform random chance.

#### Key Findings

**Overall Performance**
- **Overall alignment (55.1%)** actually **underperforms** the expected random chance (56.9%) by -1.7%
- Only **3 out of 9 method-stock combinations** beat random expectations
- The best-performing combination (GOOGL TextBlob) only exceeds random chance by 2.5%
  
**Stock-Specific Patterns**
- **GOOGL** shows the strongest but still minimal performance:
   - Alignment rates range from 62.7% to 66.7%
   - However, these rates are almost identical to what random chance would predict (63.0%-64.1%)
   - The apparent "success" is largely due to GOOGL's consistent positive returns and sentiment

- **NVDA** demonstrates near-random performance:
   - Alignment rates hover around 57-61%, barely differing from expected random levels
   - The methods show inconsistent performance, with TextBlob and Combined Sentiment slightly above and Vader below chance

- **TSLA** shows concerning underperformance:
   - All three sentiment methods **underperform** random chance by 3.2-10.8%
   - TextBlob sentiment shows the worst performance, aligning only 37.3% of the time
   - This suggests sentiment for TSLA may be systematically misleading

**Methodological Consistency**
- No sentiment method demonstrates clear superiority
- TextBlob shows the most variable performance (best for GOOGL, worst for TSLA)
- The inconsistency across stocks suggests sentiment alignment is stock-specific rather than method-driven

#### Critical Implications

The alignment results provide the final, compelling evidence against our hypothesis. Not only are correlations statistically insignificant, but sentiment fails to provide even basic directional accuracy above random chance. The fact that overall alignment underperforms random expectations suggests that using Reddit sentiment as a directional indicator would be counterproductive for investment decisions.

The results align with our earlier findings: the relationship between Reddit sentiment and stock returns is essentially non-existent in our dataset, regardless of how we measure or analyze it.

# 8. Synthesis & Interpretation

## 8.1 Hypothesis Resolution
**The evidence strongly rejects our initial hypothesis.**  
We hypothesized a "statistically significant positive correlation between weekly Reddit sentiment scores and weekly stock returns," but comprehensive testing across three analytical approaches reveals:

- **Zero statistically significant correlations** across 9 stock-method combinations
- **Correlation coefficients indistinguishable from random noise** (average r = 0.052) with wide confidence intervals
- **Alignment rates that underperform random chance expectations**, suggesting sentiment fails even as a basic directional indicator

The consistency of null findings across all three sentiment measurement methods (VADER, TextBlob, Combined) and all three technology stocks provides compelling multi-faceted evidence against the hypothesized relationship.

## 8.2 Key Findings Summary

1. **No Statistical Significance**  
All correlation p-values (0.386-0.889) far exceed the 0.05 significance threshold, with **all 95% confidence intervals including zero**

2. **Negligible Practical Impact**  
Even if correlations were significant, sentiment explains only **0.6% of return variation** on average (R² = 0.006), with the strongest relationship explaining just 1.5%

3. **Directional Inaccuracy**  
Sentiment alignment **underperforms random chance** by 1.7% overall, with only 3 of 9 method-stock combinations beating expected random alignment

4. **Methodological Consistency**  
All three sentiment analysis approaches (VADER, TextBlob, Combined) yield similarly weak results, with **identical confidence interval widths** (avg 0.548) across methods

5. **Stock-Specific Patterns**  
TSLA shows the weakest alignment (**underperforming random chance by 3.2-10.8%**), potentially reflecting its controversial nature among investors

6. **Visual Confirmation**  
Scatterplots show genuinely scattered data with **no discernible linear patterns** or influential outliers across all sentiment methods

## 8.3 Limitations & Implications

**Critical Limitations from Our Analysis**
- **Overwhelming Sentiment Bias**  
  With 76-98% of weeks showing positive sentiment, we essentially tested "degrees of optimism" rather than true sentiment direction
- **Bull Market Context**  
  The strong upward trend in all stocks during our study period created a challenging environment for detecting sentiment signals against the prevailing market momentum
- **Temporal Clustering**  
  The mid-period sentiment decline across all stocks suggests market-wide factors may have confounded stock-specific relationships
- **Platform Specificity**  
  Findings are limited to Reddit's user demographics and may not generalize to other social media platforms or investor populations

**Practical Implications**
- **For Investors**  
  Reddit sentiment provides no reliable signal for short-term trading decisions and would likely underperform simple random guessing given the alignment results
- **For Researchers**  
  Weekly aggregation may be too coarse to capture the fast-moving nature of social media sentiment's impact on markets
- **For Methodology**  
  The consistency of null results across three sentiment measurement methods suggests the issue lies not in sentiment quantification but in the fundamental relationship itself

## 8.4 Future Research Directions

1. **Higher Frequency Analysis**  
Investigate daily, hourly, or intraday relationships to capture faster sentiment-return dynamics that may be obscured by weekly aggregation, potentially revealing shorter-term relationships.

2. **Sentiment Neutralization Methods**  
Develop techniques to account for the overwhelming positivity bias in financial social media, enabling better detection of meaningful sentiment shifts amid generally optimistic discourse.

3. **Event-Study Approach**  
Focus analysis on specific market-moving events (earnings announcements, product launches, regulatory news) where sentiment may have clearer, less confounded relationships with price movements.

4. **Multi-Platform Analysis**  
Incorporate data from Twitter, StockTwits, and other financial platforms to create a more comprehensive sentiment measure and test generalizability beyond Reddit's user base.

5. **Alternative Sentiment Methodologies**  
Explore more sophisticated NLP approaches including aspect-based sentiment analysis to distinguish between discussions about different company facets (products, management, financials).

6. **Control Variable Integration**  
Incorporate known market factors (volatility indices, sector performance, macroeconomic data) to isolate sentiment's unique contribution from other market drivers.